# Пример загрузки данных Divvy Bikes

Этот notebook демонстрирует различные способы загрузки данных.

In [3]:
import sys
sys.path.append('..')

import polars as pl
from src.data.load_data import (
    load_trips, load_stations, load_daily_metrics,
    load_raw_data, DataLoader
)

## 1. Быстрая загрузка готовых данных

In [4]:
# Загрузить все поездки (использует processed/trips_final.parquet если есть)
trips_df = load_trips()

print(f"Загружено поездок: {trips_df.shape[0]:,}")
print(f"Колонок: {trips_df.shape[1]}")
print(f"\nПервые строки:")
trips_df.head()

SchemaError: type Int64 is incompatible with expected type String

## 2. Загрузка сырых данных за конкретный год

In [5]:
# Загрузить данные за 2024 год
trips_2024 = load_raw_data(year=2024)

print(f"Поездок за 2024: {trips_2024.shape[0]:,}")
print(f"\nКолонки:")
print(trips_2024.columns)

FileNotFoundError: Не найдены файлы поездок за 2024 год

## 3. Использование DataLoader для гибкой загрузки

In [6]:
loader = DataLoader()

# Загрузить данные за диапазон лет
trips_range = loader.load_raw_trips_range(2020, 2024)

print(f"Поездок за 2020-2024: {trips_range.shape[0]:,}")

# Посмотреть распределение по годам
trips_range.group_by('year').agg(
    pl.count().alias('trips_count')
).sort('year')

Пропуск 2021 года: данные не найдены
Пропуск 2022 года: данные не найдены
Пропуск 2023 года: данные не найдены
Пропуск 2024 года: данные не найдены


Поездок за 2020-2024: 426,887


/var/folders/m_/0vg5klms5wqfl8z1jkpk95tr0000gn/T/ipykernel_2759/3276194592.py:10: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias('trips_count')


year,trips_count
i32,u32
2020,426887


## 4. Загрузка промежуточных данных

In [7]:
# Загрузить очищенные данные
try:
    cleaned_trips = loader.load_interim_data('trips_cleaned')
    print(f"Очищенных поездок: {cleaned_trips.shape[0]:,}")
except FileNotFoundError:
    print("Очищенные данные еще не созданы")

Очищенные данные еще не созданы


## 5. Загрузка обработанных данных

In [8]:
# Загрузить ежедневные агрегаты
try:
    daily_df = load_daily_metrics()
    print(f"Дней с данными: {daily_df.shape[0]:,}")
    daily_df.head()
except FileNotFoundError:
    print("Ежедневные агрегаты еще не созданы")

Ежедневные агрегаты еще не созданы


## 6. Сохранение результатов

In [9]:
# Пример обработки и сохранения
from src.config.paths import INTERIM_DATA_DIR

# Простая обработка
processed = trips_2024.head(1000)  # Пример

# Сохранить результат
loader.save_dataframe(
    processed,
    INTERIM_DATA_DIR / 'example_processed.parquet'
)

print("✓ Данные сохранены")

NameError: name 'trips_2024' is not defined

## 7. Использование LazyFrame для эффективности

In [10]:
# Для больших данных используйте LazyFrame
from src.config.paths import TRIPS_FINAL

if TRIPS_FINAL.exists():
    # Ленивая загрузка
    lazy_df = pl.scan_parquet(TRIPS_FINAL)
    
    # Выполнить операции без загрузки всех данных в память
    result = (
        lazy_df
        .filter(pl.col('year') == 2024)
        .group_by('month')
        .agg(pl.count().alias('trips'))
        .collect()  # Только здесь происходит вычисление
    )
    
    print(result)
else:
    print("Финальные данные еще не созданы")

Финальные данные еще не созданы
